In this block we import gym, make an environment with the Taxi cab problem and render the same. The reset option is to make sure the environment is reset to a random configuration each time.

In [2]:
import gym
env = gym.make('Taxi-v2').env
env.reset()
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



Here I am trying to see the action space (i.e the actions the agent can take) and the observation space (i.e the number of configurations the problem can have).

In [4]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space)) # Print action space
print("State Space {}".format(env.observation_space)) # Print observation space

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


For the purpose of understanding how encoding works, I will try to encode the random state generated in the last code cell. Here the indices for R, G, Y and B are:<br>

1. R:0  
2. G:1 
3. Y:2 
4. B:3 

Puprle denotes the destination and blue the pick-up point. The first two parameters are the row and column numbers of the taxi. All numbering starts from 0 and moves from top to bottom. This means the encoding parameters are as folows (4,1,1,2)

In [7]:
state = env.encode(4, 1, 1, 2) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 426
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



The reward function defined for this environment is as follows. Any movement not resulting in dropping the passenger in the target location accrues -1 points. Any time the action pick-up or drop-off is performed falsely is punished with a -10 reward. The output of the reward function P[state_number] is a dictionary in the following format:<br>
<h3 align = "center">{action:[(probablility,next_state,reward,Goal_achieved)]}</h3> .<br>Here probability of an action is always 1. The reward distribution for state 426 seen above is listed below.

In [11]:
print(env.P[426])

{0: [(1.0, 426, -1, False)], 1: [(1.0, 326, -1, False)], 2: [(1.0, 446, -1, False)], 3: [(1.0, 426, -1, False)], 4: [(1.0, 426, -10, False)], 5: [(1.0, 426, -10, False)]}


Now instead of using Q-learning a random sampling of action space to guide the agent will be portrayed below. This is highly inefficient as displayed by the timesteps taken and penalties incurred.

In [13]:
epochs = 0
penalties,reward = 0,0
frames = []
done = False

In [14]:
while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    if reward == -10:
        penalties += 1
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )
    epochs = epochs+1
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 828
Penalties incurred: 277


828 time steps were taken with penalities of 277 (Penalties refer to any time -10 points were accrued). The frames captured is displayed as an "annimation" through the execution of the code cell below.

In [15]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 828
State: 410
Action: 5
Reward: 20


A good explanation of Q-learning is found from slide 42 onwards in CS-231 course offered by Stanford (http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture14.pdf). The update police of the Q or quality value boils down to:
<h3 align="center">Q(t+1)(state,action) = (1-alpha)(Q(t)(state,action)+alpha*(reward(t)+gamma*(Maximum reward possible from next state))</h3><br>Here alpha is learning rate and gamma is the discount factor applied to the maxium potential rewards. 

Here the Q-table that contains the quality value for each action at each state is initialized to all zeroes. The agent will be trained for 100000 epochs and the solution for the above state will be executed from this learned policy.

In [16]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [17]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.5
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 22.4 s, sys: 948 ms, total: 23.4 s
Wall time: 22.1 s


In [18]:
total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    reward_printer = 0
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1
        reward_printer += reward
        epochs += 1
    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 12.78
Average penalties per episode: 0.0


For 100 random episodes it is clear that each configuration is solved in 12.78 timesteps. A marked improvement from random spampling in the previous method.

In [24]:
frames = []
done = False
state=426
env.s = state # SOlving for state 426 that required 800 plus timesteps for solution
epochs = 0
penalties,reward = 0,0
while not done:
    action = np.argmax(q_table[state])
    state, reward, done, info = env.step(action)
    if reward == -10:
        penalties += 1
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )
    epochs = epochs+1
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 17
Penalties incurred: 0


We observe that now the number of timesteps is only 17 a lot less than the previous 828 timesteps. Also there 0 penalties incurred. Below is the proposed policy.

In [25]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 17
State: 410
Action: 5
Reward: 20
